In [2]:
import numpy as np
from numpy.core.fromnumeric import shape
import os

In [3]:
%matplotlib inline
from nuscenes.nuscenes import NuScenes

nusc = NuScenes(version='v1.0-mini', dataroot='/home/max/projects/masterthesis/mini_nuscenes', verbose=True)

Loading NuScenes tables for version v1.0-mini...
23 category,
8 attribute,
4 visibility,
911 instance,
12 sensor,
120 calibrated_sensor,
31206 ego_pose,
8 log,
10 scene,
404 sample,
31206 sample_data,
18538 sample_annotation,
4 map,
Done loading in 0.939 seconds.
Reverse indexing ...
Done reverse indexing in 0.1 seconds.


In [4]:
#Get set of scenes
scenes = nusc.scene
#Get first scenes
scene_0 = scenes[0]
# Get token of first frame
first_sample_token = scene_0['first_sample_token']
sample_0 = nusc.get('sample', first_sample_token)
# Get tokens for 2 following frames
second_sample_token = sample_0['next']
sample_1 = nusc.get('sample', second_sample_token)
third_sample_token = sample_1['next']
sample_2 = nusc.get('sample', third_sample_token)

# Get LIDAR pointcloud
sensor = 'LIDAR_TOP'
lidar_top_data_0 = nusc.get('sample_data', sample_0['data'][sensor])
# Get LIDAR KF pointcloud
lidar_top_data_1 = nusc.get('sample_data', sample_1['data'][sensor])
# Get LIDAR KF pointcloud
lidar_top_data_2 = nusc.get('sample_data', sample_2['data'][sensor])

In [5]:
#Filter out car/vehicle object
pcl0_path, boxes0, _= nusc.get_sample_data(lidar_top_data_0['token'], selected_anntokens=None, use_flat_vehicle_coordinates =False)
pcl1_path, boxes1, _= nusc.get_sample_data(lidar_top_data_1['token'], selected_anntokens=None, use_flat_vehicle_coordinates =False)
pcl2_path, boxes2, _= nusc.get_sample_data(lidar_top_data_2['token'], selected_anntokens=None, use_flat_vehicle_coordinates =False)


In [6]:
# Get point clouds
import os.path as osp
from nuscenes.utils.data_classes import LidarPointCloud, Box

# pcl0_path = osp.join(nusc.dataroot, lidar_top_data_0['filename'])
# pcl1_path = osp.join(nusc.dataroot, lidar_top_data_1['filename'])
# pcl2_path = osp.join(nusc.dataroot, lidar_top_data_2['filename'])
#Load Pointclouds
pc0 = LidarPointCloud.from_file(pcl0_path)
pc1 = LidarPointCloud.from_file(pcl1_path)
pc2 = LidarPointCloud.from_file(pcl2_path)

In [7]:
nusc.list_categories()

Category stats for split v1.0-mini:
human.pedestrian.adult      n= 4765, width= 0.68±0.11, len= 0.73±0.17, height= 1.76±0.12, lw_aspect= 1.08±0.23
human.pedestrian.child      n=   46, width= 0.46±0.08, len= 0.45±0.09, height= 1.37±0.06, lw_aspect= 0.97±0.05
human.pedestrian.constructi n=  193, width= 0.69±0.07, len= 0.74±0.12, height= 1.78±0.05, lw_aspect= 1.07±0.16
human.pedestrian.personal_m n=   25, width= 0.83±0.00, len= 1.28±0.00, height= 1.87±0.00, lw_aspect= 1.55±0.00
human.pedestrian.police_off n=   11, width= 0.59±0.00, len= 0.47±0.00, height= 1.81±0.00, lw_aspect= 0.80±0.00
movable_object.barrier      n= 2323, width= 2.32±0.49, len= 0.61±0.11, height= 1.06±0.10, lw_aspect= 0.28±0.09
movable_object.debris       n=   13, width= 0.43±0.00, len= 1.43±0.00, height= 0.46±0.00, lw_aspect= 3.35±0.00
movable_object.pushable_pul n=   82, width= 0.51±0.06, len= 0.79±0.10, height= 1.04±0.20, lw_aspect= 1.55±0.18
movable_object.trafficcone  n= 1378, width= 0.47±0.14, len= 0.45±0.07, heigh

In [8]:
category = nusc.get('sample_annotation',boxes0[16].token)['category_name']
print(category)
if( category.find('vehicle') != -1 ):
    print(category)


vehicle.car
vehicle.car


In [9]:
#Find only vehicle objects for frame 0
vehicle_boxes_tokens_0 = []
for box in boxes0:
    category = nusc.get('sample_annotation',box.token)['category_name']
    if( category.find('vehicle') != -1 ):
        vehicle_boxes_tokens_0.append(box.token)
print('shape:',np.shape(vehicle_boxes_tokens_0),'\n List:\n',vehicle_boxes_tokens_0)

shape: (13,) 
 List:
 ['924ee6ac1fed440a9d9e3720aac635a0', '36d52dfedd764b27863375543c965376', '63b89fe17f3e41ecbe28337e0e35db8e', '16140fbf143d4e26a4a7613cbd3aa0e8', '83d881a6b3d94ef3a3bc3b585cc514f8', '74986f1604f047b6925d409915265bf7', '076a7e3ec6244d3b84e7df5ebcbac637', 'cda0a9085607438c9b1ea87f4360dd64', 'a6f7d4bb60374f868144c5ba4431bf4c', '91cb8f15ed4444e99470d43515e50c1d', '26fb370c13f844de9d1830f6176ebab6', 'b7cbc6d0e80e4dfda7164871ece6cb71', '49f76277d07541c5a584aa14c9d28754']


In [10]:
#Find only car objects for frame 0
car_boxes_tokens_0 = []
car_boxes_0 = []
for box in boxes0:
    category = nusc.get('sample_annotation',box.token)['category_name']
    if( category.find('vehicle.car') != -1 ):
        car_boxes_tokens_0.append(box.token)
        car_boxes_0.append(box)
print('shape:',np.shape(car_boxes_tokens_0),'\n List:\n',car_boxes_tokens_0)

shape: (8,) 
 List:
 ['924ee6ac1fed440a9d9e3720aac635a0', '63b89fe17f3e41ecbe28337e0e35db8e', '16140fbf143d4e26a4a7613cbd3aa0e8', '74986f1604f047b6925d409915265bf7', 'cda0a9085607438c9b1ea87f4360dd64', 'a6f7d4bb60374f868144c5ba4431bf4c', '26fb370c13f844de9d1830f6176ebab6', '49f76277d07541c5a584aa14c9d28754']


In [11]:
# # Find all car annotations in nuscenes
# car_boxes_tokens_all = []
# for sample_annotation in nusc.sample_annotation:
#     category = sample_annotation['category_name']
#     if( category.find('vehicle.car') != -1 ):
#         car_boxes_tokens_all.append(sample_annotation["token"])

# print('shape:',np.shape(car_boxes_tokens_all),'\n List:\n',car_boxes_tokens_all[:5])

# # 
# _, boxes0_car, _= nusc.get_sample_data( lidar_top_data_0['token'], \
#      selected_anntokens=car_boxes_tokens_all, use_flat_vehicle_coordinates =False)

# # Not a viable option because it gets all sample annotations and not only the ones of frame 0
# print(np.shape(boxes0_car))

In [21]:
from utility import get_box_centers, filter_boxes

car_boxes_0 = filter_boxes(nusc, boxes = boxes0, categoryQuery= 'vehicle.car')
car_boxes_1 = filter_boxes(nusc, boxes = boxes1, categoryQuery= 'vehicle.car')
car_boxes_2 = filter_boxes(nusc, boxes = boxes2, categoryQuery= 'vehicle.car')

centers0 = get_box_centers(car_boxes_0)
centers1 = get_box_centers(car_boxes_1)
centers2 = get_box_centers(car_boxes_2)

# Special Shift parameter in meter
SPATIAL_SHIFT_TIMEFRAMES = 20
# Boxes 0 can stay at the current frame
# centers0
# Boxes 1 must be translated up by l meters
centers1 += np.array([0,0,SPATIAL_SHIFT_TIMEFRAMES])

# Boxes 2 must be translated up by 2*l meters
centers2 += np.array([0,0,2*SPATIAL_SHIFT_TIMEFRAMES])

# Add all centroids into one array
centers = np.empty((0,3))
centers = np.append(centers, centers0, axis=0)
centers = np.append(centers, centers1, axis=0)
centers = np.append(centers, centers2, axis=0)

In [22]:
# Build KNN Graph
from sklearn.neighbors import NearestNeighbors

#Build graph in a more organised manner
from graph.graph_generation import SpatioTemporalGraph, Timeframe

nbrs_0 = NearestNeighbors(n_neighbors=5, algorithm='ball_tree').fit(centers0)
# Frame t0
#Compute K nearest neighbors
spatial_distances_0, spatial_indices_0 = nbrs_0.kneighbors(centers0)
# Make a list of tuple pairs
spatial_pairs = [] 
for curr_node_idx ,neigborhood_indices,  in enumerate(spatial_indices_0):
    for neigbor_index in neigborhood_indices:
        spatial_pairs.append( \
            ( (Timeframe.t0,curr_node_idx) , ((Timeframe.t0,neigbor_index)) ) )

#Frame t1
nbrs_1 = NearestNeighbors(n_neighbors=5, algorithm='ball_tree').fit(centers1)
spatial_distances_1, spatial_indices_1 = nbrs_1.kneighbors(centers1)
# Make a list of tuple pairs
for curr_node_idx ,neigborhood_indices,  in enumerate(spatial_indices_1):
    for neigbor_index in neigborhood_indices:
        spatial_pairs.append( \
            ( (Timeframe.t1,curr_node_idx) , ((Timeframe.t1,neigbor_index)) ) )

#Frame t2
nbrs_2 = NearestNeighbors(n_neighbors=5, algorithm='ball_tree').fit(centers2)
spatial_distances_2, spatial_indices_2 = nbrs_2.kneighbors(centers2)
# Make a list of tuple pairs
for curr_node_idx ,neigborhood_indices,  in enumerate(spatial_indices_2):
    for neigbor_index in neigborhood_indices:
        spatial_pairs.append( \
            ( (Timeframe.t2,curr_node_idx) , ((Timeframe.t2,neigbor_index)) ) )

testgraph = SpatioTemporalGraph(spatial_pairs)

In [23]:
from utility import is_same_instance

# Add temporal edges #####################
# Only connect unique objects to themselves. Only Connect boxes
# that belong to the same object instance and subsequently same class
#Initiate graph
temporal_pairs = []
# Looking for connections between frame i and frame j
# Frame 0 ==> Frame 1
for i,box_i in enumerate(car_boxes_0):
    for j,box_j in enumerate(car_boxes_1):
        if is_same_instance(nusc, box_i.token, box_j.token):
            temporal_pairs.append( \
                ( (Timeframe.t0,i) , ((Timeframe.t1,j)) ) )
print(np.shape(temporal_pairs))
# Frame 1 ==> Frame 2
for i,box_i in enumerate(car_boxes_1):
    for j,box_j in enumerate(car_boxes_2):
        if is_same_instance(nusc, box_i.token, box_j.token):
            temporal_pairs.append( \
                ( (Timeframe.t1,i) , ((Timeframe.t2,j)) ) )
print(np.shape(temporal_pairs))
print(temporal_pairs[0])

testgraph.add_connections(temporal_pairs)

(8, 2, 2)
(18, 2, 2)
((<Timeframe.t0: 0>, 0), (<Timeframe.t1: 1>, 0))


In [24]:
# Get all spatial Edges
spatial_pointpairs0 = []
for reference_node in testgraph._graph:
    if(reference_node[0]== Timeframe.t0):
        for neighbor_node in testgraph._graph[reference_node]:
            # print(neighbor_index[0])
            timestep, idx = neighbor_node[0],neighbor_node[1]
            if timestep == Timeframe.t0:
                spatial_pointpairs0.append([reference_node[1],idx])

print(np.shape(spatial_pointpairs0))
testarray = testgraph.get_spatial_pointpairs(Timeframe.t0)
assert spatial_pointpairs0 == testarray

spatial_pointpairs1 = testgraph.get_spatial_pointpairs(Timeframe.t1)
spatial_pointpairs2 = testgraph.get_spatial_pointpairs(Timeframe.t2)

(50, 2)


In [25]:
print(np.shape(centers0))
print(np.shape(centers1))
print(np.shape(centers2))
print(np.shape(centers))
print(len(testgraph._graph))
print(len(testgraph._graph[(Timeframe.t1, 0)]))
print(testgraph._graph[(Timeframe.t0, 0)])

(8, 3)
(10, 3)
(11, 3)
(29, 3)
29
7
{(<Timeframe.t0: 0>, 5), (<Timeframe.t0: 0>, 4), (<Timeframe.t0: 0>, 0), (<Timeframe.t1: 1>, 0), (<Timeframe.t0: 0>, 6), (<Timeframe.t0: 0>, 2)}


In [26]:
# Get all temporal edges

# Get points 
def get_points(centers0, centers1 ,centers2, reference_node):
    print(reference_node)
    if(reference_node[0]== Timeframe.t0):
        return centers0[reference_node[1]]
    elif (reference_node[0]== Timeframe.t1):
        return centers1[reference_node[1]]
    elif (reference_node[0]== Timeframe.t2):
        return centers2[reference_node[1]]
    else:
        return AttributeError
        
# Get all temporal edges (only frame 0->1 and 1->2)
# global_center_list = centers.tolist()
temporal_pairs_indices = []
for reference_node in testgraph._graph:
    reference_timeframe = reference_node[0]

    # Find corresponding indices in global centers list
    point_a = get_points(centers0, centers1 ,centers2,reference_node)
    reference_idx_global = np.argwhere(centers == point_a)[0,0]

    print(len(testgraph._graph[reference_node]))
    for neighbor_node in testgraph._graph[reference_node]:
        neighbor_timeframe, neighbor_idx = neighbor_node[0],neighbor_node[1]
        if neighbor_timeframe != reference_timeframe:
            # Find corresponding indices in global centers list
            # Reference Node
            point_b = get_points(centers0, centers1 ,centers2,neighbor_node)
            neighbor_idx_global = np.argwhere(centers == point_b)[0,0]
            #Append global indices into list
            temporal_pairs_indices.append([reference_idx_global,neighbor_idx_global])


(<Timeframe.t0: 0>, 0)
6
(<Timeframe.t1: 1>, 0)
(<Timeframe.t0: 0>, 6)
6
(<Timeframe.t1: 1>, 6)
(<Timeframe.t0: 0>, 5)
9
(<Timeframe.t1: 1>, 5)
(<Timeframe.t0: 0>, 4)
9
(<Timeframe.t1: 1>, 4)
(<Timeframe.t0: 0>, 2)
8
(<Timeframe.t1: 1>, 2)
(<Timeframe.t0: 0>, 1)
6
(<Timeframe.t1: 1>, 1)
(<Timeframe.t0: 0>, 7)
7
(<Timeframe.t1: 1>, 7)
(<Timeframe.t0: 0>, 3)
7
(<Timeframe.t1: 1>, 3)
(<Timeframe.t1: 1>, 0)
7
(<Timeframe.t2: 2>, 0)
(<Timeframe.t0: 0>, 0)
(<Timeframe.t1: 1>, 6)
7
(<Timeframe.t0: 0>, 6)
(<Timeframe.t2: 2>, 6)
(<Timeframe.t1: 1>, 5)
10
(<Timeframe.t0: 0>, 5)
(<Timeframe.t2: 2>, 5)
(<Timeframe.t1: 1>, 9)
8
(<Timeframe.t2: 2>, 10)
(<Timeframe.t1: 1>, 8)
9
(<Timeframe.t2: 2>, 8)
(<Timeframe.t1: 1>, 1)
7
(<Timeframe.t0: 0>, 1)
(<Timeframe.t2: 2>, 1)
(<Timeframe.t1: 1>, 2)
8
(<Timeframe.t2: 2>, 2)
(<Timeframe.t0: 0>, 2)
(<Timeframe.t1: 1>, 7)
8
(<Timeframe.t0: 0>, 7)
(<Timeframe.t2: 2>, 7)
(<Timeframe.t1: 1>, 4)
9
(<Timeframe.t0: 0>, 4)
(<Timeframe.t2: 2>, 4)
(<Timeframe.t1: 1>, 3

In [27]:
import open3d as o3d
from open3d import geometry

############ Parameters
upward_shift = 20
##############

# Integrate All Center points in Pointcloud-object not necessary but test
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(centers)

# Include Pointcloud of timeframe 0 into visualization 
pc0_transposed = np.transpose(pc0.points)
print("Shape:",np.shape(pc0_transposed))
pointcloud0 = o3d.geometry.PointCloud()
pointcloud0.points = o3d.utility.Vector3dVector(pc0_transposed[:, :3])
#Add some color
point_color=(0.5, 0.5, 0.5)
points_colors = np.tile(np.array(point_color), (pc0_transposed.shape[0], 1))
pointcloud0.colors = o3d.utility.Vector3dVector(points_colors)

# Include Pointcloud of timeframe 1 into visualization 
pc1_transposed = np.transpose(pc1.points)
print("Shape:",np.shape(pc1_transposed))
pointcloud1 = o3d.geometry.PointCloud()
pointcloud1.points = o3d.utility.Vector3dVector(pc1_transposed[:, :3])
#Add some color
point_color=(0.5, 0.5, 0.5)
points_colors = np.tile(np.array(point_color), (pc1_transposed.shape[0], 1))
pointcloud1.colors = o3d.utility.Vector3dVector(points_colors)
# Translate up to stack the point clouds
pointcloud1.translate(np.array([0,0,upward_shift]))

# Include Pointcloud of timeframe 2 into visualization 
pc2_transposed = np.transpose(pc2.points)
print("Shape:",np.shape(pc2_transposed))
pointcloud2 = o3d.geometry.PointCloud()
pointcloud2.points = o3d.utility.Vector3dVector(pc2_transposed[:, :3])
#Add some color
point_color=(0.5, 0.5, 0.5)
points_colors = np.tile(np.array(point_color), (pc2_transposed.shape[0], 1))
pointcloud2.colors = o3d.utility.Vector3dVector(points_colors)
# Translate up to stack the point clouds
pointcloud2.translate(np.array([0,0,upward_shift * 2]))

# Include reference frame
mesh_frame = geometry.TriangleMesh.create_coordinate_frame(
            size=1, origin=[0, 0, 0])  # create coordinate frame

# Draw Graph/Edges with Lineset
# Spatial Edges Red Edges
colors = [[1, 0, 0] for i in range(len(spatial_pointpairs0))]
line_set0 = geometry.LineSet(points=o3d.utility.Vector3dVector(centers0),
    lines=o3d.utility.Vector2iVector(spatial_pointpairs0),)
line_set0.colors = o3d.utility.Vector3dVector(colors)

colors = [[1, 0, 0] for i in range(len(spatial_pointpairs1))]
line_set1 = geometry.LineSet(points=o3d.utility.Vector3dVector(centers1),
    lines=o3d.utility.Vector2iVector(spatial_pointpairs1),)
line_set1.colors = o3d.utility.Vector3dVector(colors)

colors = [[1, 0, 0] for i in range(len(spatial_pointpairs2))]
line_set2 = geometry.LineSet(points=o3d.utility.Vector3dVector(centers2),
    lines=o3d.utility.Vector2iVector(spatial_pointpairs2),)
line_set2.colors = o3d.utility.Vector3dVector(colors)

# Temporal Edges 
colors = [[0, 0, 1] for i in range(len(temporal_pairs_indices))]
temporal_line_set = geometry.LineSet(points=o3d.utility.Vector3dVector(centers),
    lines=o3d.utility.Vector2iVector(temporal_pairs_indices),)
temporal_line_set.colors = o3d.utility.Vector3dVector(colors)

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.
Shape: (34688, 4)
Shape: (34720, 4)
Shape: (34720, 4)


In [35]:
rot_axis = 2
bbox_color=(0, 1, 0)
line_set_bounding_boxes_0 = []
for box in car_boxes_0:
    center = box.center
    dim = box.wlh
    yaw = np.zeros(3)
    yaw[rot_axis] = box.orientation.angle
    rot_mat = o3d.geometry.get_rotation_matrix_from_xyz(yaw)
    center[rot_axis] += dim[
                rot_axis] / 2  # bottom center to gravity center

    box3d = geometry.OrientedBoundingBox(center, rot_mat, dim)

    line_set_bounding_box_0 = o3d.geometry.LineSet.create_from_oriented_bounding_box(box3d)
    line_set_bounding_box_0.paint_uniform_color(bbox_color)
    line_set_bounding_boxes_0.append(line_set_bounding_box_0)

#     # change the color of points which are in box
#     if pcd is not None and mode == 'xyz':
#         indices = box3d.get_point_indices_within_bounding_box(pcd.points)
#         points_colors[indices] = in_box_color

# # update points colors
#     if pcd is not None:
#         pcd.colors = o3d.utility.Vector3dVector(points_colors)
#         vis.update_geometry(pcd)

In [37]:
geometry_list = [pcd,mesh_frame,line_set0, \
            line_set1,line_set2,temporal_line_set,\
            pointcloud0,pointcloud1, pointcloud2]
geometry_list += line_set_bounding_boxes_0
o3d.visualization.draw_geometries(geometry_list )

In [33]:
o3d.visualization.draw_geometries([line_set_bounding_boxes_0])